In [76]:
import numpy as np

data = np.loadtxt('./data/input.data')

print(data.shape)
print(data[:5])

X_train = data[:, 0].reshape(-1, 1)  # 特徵
y_train = data[:, 1].reshape(-1, 1)   # 標籤



(34, 2)
[[-50.18038796   1.77474681]
 [-48.33784599   1.13053693]
 [-43.0202886   -2.47591107]
 [-41.29017074  -0.66589532]
 [-38.06586017  -0.47291983]]


In [77]:
# import pandas as pd

# X_train = pd.read_csv('./data/X_train.csv', header=None)
# X_test = pd.read_csv('./data/X_test.csv', header=None)
# y_train = pd.read_csv('./data/Y_train.csv', header=None)
# y_test = pd.read_csv('./data/Y_test.csv', header=None)

# print("X_train shape: ",X_train.shape)
# print("X_test shape: ",X_test.shape)
# print("y_train shape: ",y_train.shape)
# print("y_test shape: ",y_test.shape)


In [78]:
import numpy as np
import matplotlib.pyplot as plt

# Rational Quadratic kernel 定義
def rational_quadratic_kernel(X1, X2, length_scale=1.0, alpha=1.0, variance=1.0):
    sqdist = np.sum(X1**2, axis=1).reshape(-1,1) + np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T)
    return variance * (1 + sqdist / (2 * alpha * length_scale**2)) ** (-alpha)

# Gaussian Process Regression 函數
def gaussian_process(X_train, y_train, X_test, length_scale=1.0, alpha=1.0, variance=1.0, noise=1e-10):

    X_train = np.asarray(X_train).reshape(-1, 1)
    y_train = np.asarray(y_train).ravel()
    X_test = np.asarray(X_test).reshape(-1, 1)

    # 計算核矩陣
    K = rational_quadratic_kernel(X_train, X_train, length_scale, alpha, variance) + noise * np.eye(len(X_train))
    K_s = rational_quadratic_kernel(X_train, X_test, length_scale, alpha, variance)
    K_ss = rational_quadratic_kernel(X_test, X_test, length_scale, alpha, variance)

    K_inv = np.linalg.inv(K)

    mu = K_s.T @ K_inv @ y_train
    cov = K_ss - K_s.T @ K_inv @ K_s

    return mu.ravel(), cov

length_scale = 1.0
alpha = 1.0
variance = 1.0
noise = 1/5

X_pred = np.linspace(-60, 60, 1000).reshape(-1, 1)
mu, cov = gaussian_process(X_train, y_train, X_pred,
                           length_scale=length_scale, alpha=alpha,
                           variance=variance, noise=noise)

plt.figure(figsize=(10, 6))
plt.scatter(X_train, y_train, color='red', label='Training data')
plt.plot(X_pred, mu, color='blue', label='Mean prediction')
plt.fill_between(X_pred.ravel(),
                 mu - 1.96 * np.sqrt(np.diag(cov)),
                 mu + 1.96 * np.sqrt(np.diag(cov)),
                 alpha=0.2, label='95% confidence interval')
plt.title('Rational Quadratic Kernel Regression')
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim(-60, 60)
plt.legend()
plt.grid(True)
plt.show()


In [79]:
from scipy.optimize import minimize
import numpy as np

def negative_log_likelihood(log_params, X_train, y_train):
    length_scale, alpha, variance, noise = np.exp(log_params)
    K = rational_quadratic_kernel(X_train, X_train, length_scale, alpha, variance) + np.eye(len(X_train)) * noise

    try:
        L = np.linalg.cholesky(K)
    except np.linalg.LinAlgError:
        return np.inf  # matrix not positive definite

    alpha_vec = np.linalg.solve(L.T, np.linalg.solve(L, y_train))
    log_det = 2 * np.sum(np.log(np.diag(L)))
    nll = 0.5 * y_train.T @ alpha_vec + 0.5 * log_det + 0.5 * len(y_train) * np.log(2 * np.pi)
    return nll

init_log_params = np.log([1.0, 1.0, 1.0, 1e-1])  # [length_scale, alpha, variance, noise]

res = minimize(negative_log_likelihood,
               init_log_params,
               args=(X_train, y_train),
               method='L-BFGS-B')


opt_length_scale, opt_alpha, opt_variance, opt_noise = np.exp(res.x)

length_scale = opt_length_scale
alpha = opt_alpha
variance = opt_variance
noise = opt_noise

X_pred = np.linspace(-60, 60, 1000).reshape(-1, 1)

mu, cov = gaussian_process(X_train, y_train, X_pred,
                           length_scale=length_scale, alpha=alpha,
                           variance=variance, noise=noise)

plt.figure(figsize=(10, 6))
plt.scatter(X_train, y_train, color='red', label='Training data')
plt.plot(X_pred, mu, color='blue', label='Mean prediction')
plt.fill_between(X_pred.ravel(),
                 mu - 1.96 * np.sqrt(np.diag(cov)),
                 mu + 1.96 * np.sqrt(np.diag(cov)),
                 alpha=0.2, label='95% confidence interval')
plt.title('Rational Quadratic Kernel Regression')
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim(-60, 60)
plt.legend()
plt.grid(True)
plt.show()

